In [43]:
import random
import numpy as np

B = 3                      #tipos de veiculos
R = 4                       #conjunto de rotas
FT = [40,28,45]              #FT_b: frota disponível por tipo de veículo
PD = [50,60,55,10]             #demanda de passageiros por hora na rota r
cap = [39,42,48]                #capacidade de passageiros para cada tipo de veiculo b
frequencia = np.zeros((B,R))     #F_br: frequência de serviço do tipo de veículo b na rota r 
Q = np.zeros((B,R))               #numero de passageiros alocados em cada rota r pelos diferentes tipos de veiculos b
rotas = []                         #lista de rotas restantes
S = 5                               #numero de soluções geradas

#variavel auxiliar p(passageiros) recebe a demanda de passageiros por hora na rota r
p = PD
#variavel auxiliar c(capacidade) recebe a capacidade de passageiros para cada tipo de veiculo b
c = cap

#imprimindo os parâmetros
print("\n Rotas = ", R, "\n Tipo de veiculo = ", B)
print(" Frota = ", FT, "\n Demanda de passageiros = ", PD, "\n Capacidade do veiculo = ", cap, "\n")

#gerando uma solução inicial
#para cada solucao s
for s in range(S):
    #para cada tipo de veiculo b
    for b in range(B):
        #numero maximo de veiculos disponiveis para cada tipo b 
        max_v = FT[b]

        #para cada rota r
        for r in range (R):
            #adiciona o conjunto de rotas na lista de rotas restantes
            rotas.append(r) 

        #enquanto houver rotas restantes
        while(rotas):
            #sorteia uma rota r
            r = random.choice(rotas)
            #pega um valor que esteja entre a capacidade de passageiros e a demanda de passageiros
            min_value = min(c[b],p[r])
            #sorteia um numero de veiculo enviado para a rota r 
            frequencia[b][r] = random.randint(0,max_v)
            #se nenhum veiculo foi enviado para rota r, a demanda de passageiros será igual a zero
            if frequencia[b][r] == 0:
                #remove a rota r da lista de rotas restantes
                rotas.remove(r)
                continue
            #sorteia um numero de passageiros para cada tipo de veiculo b na rota r
            Q[b][r] = random.randint(0, min_value)
            #atualiza a capacidade de passageiros b
            c[b] -= Q[b][r]
            #atualiza a demanda de passageiros da rota r
            p[r] -= Q[b][r]
            #atualiza o numero maximo de veiculos disponiveis para cada tipo b 
            max_v -= frequencia[b][r]
            #remove a rota r da lista de rotas restantes
            rotas.remove(r)

    #imprimindo a solução inicial
    print("_"*48)
    print("Solucao ", s+1, "\n")
    #para cada tipo de veiculo b
    for b in range(B):
        print("_Tipo ", b)
        #para cada rota r
        for r in range(R): 
            print("rota ", r, ": ", "Freq_servico: ", frequencia[b][r],"  -  demanda: ", Q[b][r] )
        print()






 Rotas =  4 
 Tipo de veiculo =  3
 Frota =  [40, 28, 45] 
 Demanda de passageiros =  [50, 60, 55, 10] 
 Capacidade do veiculo =  [39, 42, 48] 

________________________________________________
Solucao  1 

_Tipo  0
rota  0 :  Freq_servico:  12.0   -  demanda:  10.0
rota  1 :  Freq_servico:  1.0   -  demanda:  11.0
rota  2 :  Freq_servico:  11.0   -  demanda:  11.0
rota  3 :  Freq_servico:  11.0   -  demanda:  1.0

_Tipo  1
rota  0 :  Freq_servico:  8.0   -  demanda:  22.0
rota  1 :  Freq_servico:  1.0   -  demanda:  0.0
rota  2 :  Freq_servico:  1.0   -  demanda:  0.0
rota  3 :  Freq_servico:  17.0   -  demanda:  5.0

_Tipo  2
rota  0 :  Freq_servico:  24.0   -  demanda:  13.0
rota  1 :  Freq_servico:  13.0   -  demanda:  21.0
rota  2 :  Freq_servico:  2.0   -  demanda:  2.0
rota  3 :  Freq_servico:  6.0   -  demanda:  1.0

________________________________________________
Solucao  2 

_Tipo  0
rota  0 :  Freq_servico:  2.0   -  demanda:  0.0
rota  1 :  Freq_servico:  2.0   -  demanda